In [2]:
import os
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

IMG_SIZE = 128
BATCH_SIZE = 32
DATASET_DIR = "./Downloads/Data/train2"


In [4]:
file_paths = []
labels = []
class_names = sorted(os.listdir(DATASET_DIR))

for label_index, class_name in enumerate(class_names):
    class_dir = os.path.join(DATASET_DIR, class_name)
    for fname in os.listdir(class_dir):
        if fname.lower().endswith(('.jpg', '.jpeg', '.png')):
            file_paths.append(os.path.join(class_dir, fname))
            labels.append(label_index)

file_paths = np.array(file_paths)
labels = np.array(labels)


In [6]:
X_train, X_temp, y_train, y_temp = train_test_split(file_paths, labels, test_size=0.3, stratify=labels, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)


In [8]:
def process_image(file_path, label):
    image = tf.io.read_file(file_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
    image = image / 255.0
    return image, label

def create_dataset(paths, labels):
    ds = tf.data.Dataset.from_tensor_slices((paths, labels))
    ds = ds.map(process_image, num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.shuffle(8000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
    return ds

train_ds = create_dataset(X_train, y_train)
val_ds = create_dataset(X_val, y_val)
test_ds = create_dataset(X_test, y_test)

NUM_CLASSES = len(class_names)


In [10]:
from tensorflow.keras.applications import ResNet152V2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Input
from tensorflow.keras.models import Model

# Load base model
base_model = ResNet152V2(
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False  # Freeze for transfer learning

# Build model
inputs = Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)
outputs = Dense(NUM_CLASSES, activation='softmax')(x)

model = Model(inputs, outputs)

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


234545216/234545216 ━━━━━━━━━━━━━━━━━━━━ 187s 1us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 128, 128, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ resnet152v2 (Functional)             │ (None, 4, 4, 2048)          │      58,331,648 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 2048)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 2048)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │         262,272 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 25)                  │           3,225 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 58,597,145 (223.53 MB)

 Trainable params: 265,497 (1.01 MB)

 Non-trainable params: 58,331,648 (222.52 MB)

In [12]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)


Epoch 1/10
110/110 ━━━━━━━━━━━━━━━━━━━━ 148s 1s/step - accuracy: 0.2347 - loss: 2.9990 - val_accuracy: 0.6760 - val_loss: 1.1436
Epoch 2/10
110/110 ━━━━━━━━━━━━━━━━━━━━ 127s 1s/step - accuracy: 0.6125 - loss: 1.2516 - val_accuracy: 0.7613 - val_loss: 0.7985
Epoch 3/10
110/110 ━━━━━━━━━━━━━━━━━━━━ 104s 939ms/step - accuracy: 0.7058 - loss: 0.8893 - val_accuracy: 0.7893 - val_loss: 0.6997
Epoch 4/10
110/110 ━━━━━━━━━━━━━━━━━━━━ 107s 965ms/step - accuracy: 0.7658 - loss: 0.7074 - val_accuracy: 0.8173 - val_loss: 0.5963
Epoch 5/10
110/110 ━━━━━━━━━━━━━━━━━━━━ 106s 955ms/step - accuracy: 0.8193 - loss: 0.5704 - val_accuracy: 0.8120 - val_loss: 0.5743
Epoch 6/10
110/110 ━━━━━━━━━━━━━━━━━━━━ 110s 989ms/step - accuracy: 0.8256 - loss: 0.5187 - val_accuracy: 0.8333 - val_loss: 0.5214
Epoch 7/10
110/110 ━━━━━━━━━━━━━━━━━━━━ 109s 984ms/step - accuracy: 0.8333 - loss: 0.4856 - val_accuracy: 0.8347 - val_loss: 0.5210
Epoch 8/10
110/110 ━━━━━━━━━━━━━━━━━━━━ 111s 1s/step - accuracy: 0.8695 - loss: 0.

In [14]:
loss, acc = model.evaluate(test_ds)
print(f"✅ Test Accuracy: {acc:.4f}")

y_true, y_pred = [], []
for images, labels in test_ds:
    preds = model.predict(images)
    y_true.extend(labels.numpy())
    y_pred.extend(np.argmax(preds, axis=1))

print(classification_report(y_true, y_pred, target_names=class_names))


24/24 ━━━━━━━━━━━━━━━━━━━━ 18s 747ms/step - accuracy: 0.8438 - loss: 0.4890
✅ Test Accuracy: 0.8455
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 825ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 838ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 833ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 802ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 834ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 808ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 805ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 920ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 892ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 809ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 813ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 827ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 817ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 838ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 794ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 786ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 774ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 777ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 792ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 810ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 839ms/step
1/1 ━━━━━

In [16]:
model.save("plant_disease_resnet152v2.keras", save_format="keras_v3")


In [20]:
import tensorflow as tf
import numpy as np
from PIL import Image

# Load trained model
model = tf.keras.models.load_model("plant_disease_resnet152v2.keras")

# Load class names (must match training order)
class_names = [
    'Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy',
    'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot', 'Corn_(maize)___Common_rust_',
    'Corn_(maize)___healthy', 'Corn_(maize)___Northern_Leaf_Blight',
    'Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___healthy',
    'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Potato___Early_blight',
    'Potato___healthy', 'Potato___Late_blight', 'Tomato___Bacterial_spot',
    'Tomato___Early_blight', 'Tomato___healthy', 'Tomato___Late_blight',
    'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot',
    'Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___Target_Spot',
    'Tomato___Tomato_mosaic_virus', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus'
]


In [22]:
def predict_image(image_path):
    image = Image.open(image_path).convert("RGB")
    image = image.resize((128, 128))  # Match model input size
    image_array = tf.keras.preprocessing.image.img_to_array(image)
    image_array = np.expand_dims(image_array, axis=0)  # Add batch dimension
    image_array = image_array / 255.0  # Normalize

    predictions = model.predict(image_array)
    predicted_class = class_names[np.argmax(predictions)]
    confidence = np.max(predictions)

    print(f"✅ Predicted Class: {predicted_class}")
    print(f"🔍 Confidence: {confidence:.2f}")
    return predicted_class, confidence


In [24]:
image_path = "Copy of RS_Rust 2754_flipLR.JPG"  # Change this to your image path
predict_image(image_path)


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
✅ Predicted Class: Corn_(maize)___Common_rust_
🔍 Confidence: 0.90


('Corn_(maize)___Common_rust_', 0.90235275)

In [1]:
from tensorflow.keras.utils import plot_model


In [3]:
plot_model(model, to_file="resnet_model.png", show_shapes=True, show_layer_names=True)


NameError: name 'model' is not defined

In [30]:
pip install pydot

Note: you may need to restart the kernel to use updated packages.


In [1]:
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import plot_model
from IPython.display import Image  # For Jupyter Notebook display

# Step 1: Load the .keras model
model = load_model("plant_disease_resnet152v2.keras")

# Step 2: Generate architecture plot
plot_model(model, to_file="model_architecture.png", show_shapes=True, show_layer_names=True)



You must install graphviz (see instructions at https://graphviz.gitlab.io/download/) for `plot_model` to work.


In [9]:
pip install pydot graphviz


Note: you may need to restart the kernel to use updated packages.
